In [24]:
#%reset
import numpy as np
#import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import text
%matplotlib inline
#from scipy.optimize import fsolve
from enum import Enum
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Layout
#from ipywidgets.embed import embed_minimal_html
#from mpl_toolkits import mplot3d
from matplotlib.backends.backend_pdf import PdfPages #For printing to PDF
from IPython.display import FileLink,FileLinks
#import os


g = 9.81 # [m/s]
GP_BEND = 100 #Number of gridpoints in a bend

global RHO_WET, RHO_DRY, MU_WET_PIPE, MU_DRY_PIPE, MU_WET_ROLLER, MU_DRY_ROLLER, T_INPUT
RHO_WET = 50
RHO_DRY = 100
MU_WET_PIPE = 0.25 #[-]
MU_DRY_PIPE = 0.3
MU_WET_ROLLER = 0.05
MU_DRY_ROLLER = 0.1
T_INPUT = 10000

outDebug = widgets.Output(layout = Layout(width = '100%',border = '1px solid grey'))

def determineFrictionFactor(isWet,type):
    if (isWet):
        if type == SectionType.PIPE: return MU_WET_PIPE
        elif type == SectionType.ROLLER: return MU_WET_ROLLER
    else:
        if type == SectionType.PIPE: return MU_DRY_PIPE
        elif type == SectionType.ROLLER: return MU_DRY_ROLLER

def determineWeight(isWet):
    if (isWet): return RHO_WET
    else: return RHO_DRY

def rotate2global(vec,yaw,pitch): #Rotating a vector from local coordinate system to global coordinate system with rotation beta around z-axis
    cy, sy = np.cos(yaw), np.sin(yaw)
    cp, sp = np.cos(pitch), np.sin(pitch)
    R_yaw = np.array([[cy, -sy, 0], [sy, cy, 0] , [0, 0, 1]]) #3D Rotation matrix
    R_pitch = np.array([[cp, 0, sp], [0, 1, 0] , [-sp, 0, cp]]) #3D Rotation matrix
    return R_yaw @ R_pitch @ vec

# Classes
#region

class SectionType(Enum):
    PIPE = 1
    ROLLER = 2

class Section:
    def __init__(self,isWet,type):
        ###with outDebug: print("Section.__init__()")
        self.isWet = isWet
        self.type = type
        self.unrotatedPoints = self.getUnrotatedGeometry() #Local vector of 3D points in reference to unrotated coordinate system
        self.rotatedPoints = self.getRotatedGeometry() #Local vector of 3D points in reference to rotated coordinate system to match world coords
        self.rho = determineWeight(isWet)
        self.mu = determineFrictionFactor(isWet,type)
        self.inputTension = None #To be determined by Pull-in class

    def length(self):
        pass

    def plotSideGeometry(self,p0,isOddSection): #p0 is global coordinate of local coordinate system
        plt.figure(1)
        globalPoints = p0 + self.rotatedPoints
        xVecGlobal = [p[0] for p in globalPoints]
        zVecGlobal = [p[2] for p in globalPoints]
        plt.plot(xVecGlobal ,zVecGlobal,color=self.plotColor(isOddSection), ls=self.plotStyle()) #From side
        plt.plot(xVecGlobal[0],zVecGlobal[0], marker= ".", color = "k" )
        plt.plot(xVecGlobal[-1],zVecGlobal[-1], marker= ".", color = "k")
        plt.title("Geometry side view")
    
    def plotTopGeometry(self,p0,isOddSection):
        plt.figure(2)
        globalPoints = p0 + self.rotatedPoints
        xVecGlobal = [p[0] for p in globalPoints]
        yVecGlobal = [p[1] for p in globalPoints]
        plt.plot(xVecGlobal ,yVecGlobal,color=self.plotColor(isOddSection),ls=self.plotStyle()) #From top
        plt.plot(xVecGlobal[0],yVecGlobal[0], marker= ".", color = "k" )
        plt.plot(xVecGlobal[-1],yVecGlobal[-1], marker= ".", color = "k")
        plt.title("Geometry top view")

    def plot3dGeometry(self,p0,ax,col):
        globalPoints = p0 + self.rotatedPoints
        xVecGlobal = [p[0] for p in globalPoints]
        yVecGlobal = [p[1] for p in globalPoints]
        zVecGlobal = [p[2] for p in globalPoints]
        ax.plot3D(xVecGlobal, yVecGlobal, zVecGlobal, color=col) #From top
        #ax.view_init(elev=90, azim=0)

    def getUnrotatedGeometry(): #returns x and y point in local coordinates
        pass
    
    def getRotatedGeometry(self):
        zeroVec = np.zeros(3)
        rotPoints = [zeroVec for i in range(len(self.unrotatedPoints))]
        for i,p in enumerate(self.unrotatedPoints):
            rotPoints[i] = rotate2global(p,self.csYaw,self.csPitch)
        return rotPoints
    
    def getRotatedEndPoint(self):
        return self.rotatedPoints[-1] #Overwritten in TopBend class

    def minOutputTension():
        pass

    def getLvec():
        pass

    def T_vec():
        pass
    
    def getUnitInputVec(self):
        u = self.rotatedPoints[0] - self.rotatedPoints[1]
        return u/np.linalg.norm(u)

    def plotInputTensionArrowSide(self,p0):
        u_norm = (self.rotatedPoints[0] - self.rotatedPoints[1])/(np.linalg.norm(self.rotatedPoints[0] - self.rotatedPoints[1]))
        xlimRange = plt.gca().get_xlim()
        ylimRange = plt.gca().get_ylim()
        scl = 0.1*max(xlimRange[1]-xlimRange[0],ylimRange[1]-ylimRange[0])
        plt.arrow(p0[0],p0[2],scl*u_norm[0],scl*u_norm[2], head_width = scl*0.2, color="k") #Plot in opposite direction of second point
        #plt.text(p0[0] + 1.5*scl*u_norm[0], p0[2] + 1.5*scl*u_norm[2], r'$T_{in}$',va='center',ha='center')
    
    def plotOutputTensionArrowSide(self,p0):
        u_norm = (self.rotatedPoints[-1] - self.rotatedPoints[-2])/(np.linalg.norm(self.rotatedPoints[-1] - self.rotatedPoints[-2]))
        pEnd = p0 + self.rotatedPoints[-1]
        xlimRange = plt.gca().get_xlim()
        ylimRange = plt.gca().get_ylim()
        scl = 0.2*max(xlimRange[1]-xlimRange[0],ylimRange[1]-ylimRange[0])
        plt.arrow(pEnd[0],pEnd[2],scl*u_norm[0],scl*u_norm[2],  head_width = scl*0.1, color="k") #Plot in opposite direction of second point
        #plt.text(pEnd[0] + 1.25*scl*u_norm[0], pEnd[2] + 1.25*scl*u_norm[2], r'$T_{out}$',va='center',ha='center')

    def plotInputTensionArrowTop(self,p0):
        u_norm = (self.rotatedPoints[0] - self.rotatedPoints[1])/(np.linalg.norm(self.rotatedPoints[0] - self.rotatedPoints[1]))
        xlimRange = plt.gca().get_xlim()
        ylimRange = plt.gca().get_ylim()
        scl = 0.1*max(xlimRange[1]-xlimRange[0],ylimRange[1]-ylimRange[0])
        plt.arrow(p0[0],p0[1],scl*u_norm[0],scl*u_norm[1],  head_width = scl*0.2, color="k") #Plot in opposite direction of second point
        #plt.text(p0[0] + scl*u_norm[0],p0[1] + scl*u_norm[1], r'$T_{in}$',va='bottom',ha='right')
    
    def plotOutputTensionArrowTop(self,p0):
        u_norm = (self.rotatedPoints[-1] - self.rotatedPoints[-2])/(np.linalg.norm(self.rotatedPoints[-1] - self.rotatedPoints[-2]))
        pEnd = p0 + self.rotatedPoints[-1]
        xlimRange = plt.gca().get_xlim()
        ylimRange = plt.gca().get_ylim()
        scl = 0.2*max(xlimRange[1]-xlimRange[0],ylimRange[1]-ylimRange[0])
        plt.arrow(pEnd[0],pEnd[1],scl*u_norm[0],scl*u_norm[1],  head_width = scl*0.1, color="k") #Plot in opposite direction of second point
        #plt.text(pEnd[0] + scl*u_norm[0],pEnd[1] + scl*u_norm[1], r'$T_{out}$',va='bottom',ha='right')
    
    def getOutputAngle(self):
        pass
    
    def plotColor(self, isOddSectionNo):
        # if self.isWet:
        #     if isOddSectionNo: return "blue"
        #     else: return "royalblue"
        # else: 
        #     if isOddSectionNo: return "green"
        #     else: return "limegreen"
        if self.isWet: return "blue"
        else: return "deepskyblue"
    
    def plotStyle(self):
        if self.type == SectionType.PIPE: return "-"
        elif self.type == SectionType.ROLLER: return ":"
        else: raise NameError("Invalid SectionType")

class Straight(Section):
    def __init__(self,isWet,type,horDisp,vertDisp,yaw_in_deg):
        self.theta = np.arctan2(vertDisp,horDisp) #Constant pitch. Negative theta corresponds to positive pitch
        self.csPitch = -self.theta #Pitch of local coordinate system
        self.csYaw = np.deg2rad(yaw_in_deg)
        self.horDisp = horDisp
        self.vertDisp = vertDisp
        self.theta_in = self.theta
        self.theta_out = self.theta
        self.psi_in = self.csYaw
        self.psi_out = self.csYaw
        self.R = r"$\infty$"
        super().__init__(isWet,type)

    def getUnrotatedGeometry(self):
        p1 = np.array([0,0,0])
        p2 = np.array([self.length(),0,0])
        return [p1,p2] #Return list of 3D point vectors in relation to local coordinate system

    def getLvec(self):
        return 0,self.length()
    
    def length(self):
        return np.sqrt(self.horDisp**2 + self.vertDisp**2)
    
    def minOutputTension(self, inputTension):
        return inputTension + self.rho*self.length()*g*(self.mu*np.cos(self.theta) + np.sin(self.theta))
    
    def T_vec(self,inputTension):
        return inputTension,self.minOutputTension(inputTension)

class VerticalBend(Section):
    def __init__(self,isWet,type,R,theta_in_deg,theta_out_deg,yaw_in_deg):
        self.csYaw = np.deg2rad(yaw_in_deg)
        self.R = R
        self.theta_in = np.deg2rad(theta_in_deg)
        self.theta_out = np.deg2rad(theta_out_deg)
        self.thetaVec = np.linspace(self.theta_in,self.theta_out,GP_BEND)
        super().__init__(isWet,type) #Calls Section constructor
        self.L_vec = self.getLvec()
        self.psi_in = self.csYaw
        self.psi_out = self.csYaw

    def getUnrotatedGeometry(self):
        pass

    def getLvec(self): #Length along arc as a function of thetaVec
        return ((self.thetaVec-self.theta_in)*self.R)

    def length(self):
        return (self.getLvec()[-1])

    def minOutputTension(self, inputTension):
        T_theta = self.T_vec(inputTension)
        return T_theta[-1]
    
    def T_vec():
        pass
    
    def getRotatedGeometry(self):
        return super().getRotatedGeometry()

class BottomBend(VerticalBend):
    def __init__(self,isWet,type,R,theta_in_deg,theta_out_deg,psi_in_deg):
        self.csPitch = np.deg2rad(-theta_in_deg) #Pitch of local coordinate system¨
        super().__init__(isWet,type,R,theta_in_deg,theta_out_deg,psi_in_deg)
        self.theta_crits = None #To be determined with inputTension from Pull-in class
    
    def getUnrotatedGeometry(self):
        deltaThetaVec = self.thetaVec - self.theta_in
        zeroVec = np.zeros(3)
        unrotPoints = [zeroVec for i in range(len(deltaThetaVec))]
        for i,dTheta in enumerate(deltaThetaVec):
            unrotPoints[i] = np.array([0,0,self.R]) + rotate2global(np.array([0,0,-self.R]),0,-dTheta) #Pitch = -theta
        return unrotPoints

    def critLenghts(self):
        valid_theta_crits = self.theta_crits[ (self.theta_crits > self.theta_in) & (self.theta_crits < self.theta_out) ]
        L_crits = ((valid_theta_crits-self.theta_in)*self.R)
        return L_crits[L_crits > 0]

    def T_bottom(self,T_in,theta_in,theta):
        A = self.rho*self.R*g/(self.mu**2+1)
        B = self.mu**2 - 1
        C = self.mu*2
        
        exp_term = np.exp(self.mu*(theta - theta_in))

        T_bottom = T_in*(1/exp_term) + \
                 + A*( B*(np.cos(theta) - np.cos(theta_in)*(1/exp_term)) + \
                       C*(np.sin(theta) - np.sin(theta_in)*(1/exp_term)) )
        return T_bottom
    
    def T_top(self,T_in,theta_in,theta):
        A = self.rho*self.R*g/(self.mu**2+1)
        B = self.mu**2 - 1
        C = self.mu*2
        
        exp_term = np.exp(self.mu*(theta - theta_in))

        T_top = T_in*exp_term + \
                 + A*( B*(np.cos(theta) - np.cos(theta_in)*exp_term ) + \
                       C*(-np.sin(theta) + np.sin(theta_in)*exp_term) )

        return T_top

    def thetaHangoff(self,T_in):
        #N_bottom = lambda theta: self.rho*self.R*g*np.cos(theta) - self.T_bottom(T_in,self.theta_in,theta)
        #roots = fsolve(N_bottom,np.array([-np.pi/8,np.pi/8])) #Find roots from both sides
        #plt.plot(self.L_vec,N_bottom(self.thetaVec))
        thetaRange = np.linspace(-np.pi/2,np.pi/2,GP_BEND)
        N_bottom = self.rho*self.R*g*np.cos(thetaRange) - self.T_bottom(T_in,self.theta_in,thetaRange)
        roots = np.array([])
        
        for i in range(1,len(N_bottom)):
            if (N_bottom[i]*N_bottom[i-1] < 0): 
                roots = np.append(roots,thetaRange[i])
        #roots = fsolve(N_bottom,np.array([-np.pi/8,np.pi/8])) #Find roots from both sides

        return roots
    
    def T_vec(self,inputTension):
        print("T_vec called")
        self.theta_crits = self.thetaHangoff(inputTension) #gives two roots which may be outside theta_range
        if len(self.theta_crits) == 0:
            return self.T_top(inputTension,self.theta_in,self.thetaVec)
        elif len(self.theta_crits) == 2:
            isNotResting1 = (self.thetaVec <= self.theta_crits[0])
            isResting = (self.thetaVec > self.theta_crits[0]) & (self.thetaVec < self.theta_crits[1]) #Boolean values for resting theta
            isNotResting2 = (self.thetaVec >= self.theta_crits[1])

            theta_topside1 = self.thetaVec[isNotResting1]
            theta_resting = self.thetaVec[isResting]
            theta_topside2 = self.thetaVec[isNotResting2]

            # Avoid indexing empty array
            if len(theta_topside1) == 0: theta_in_topside1 = []
            else: theta_in_topside1 = theta_topside1[0]

            if len(theta_resting) == 0: theta_in_resting = []
            else: theta_in_resting = theta_resting[0]

            if len(theta_topside2) == 0: theta_in_topside2 = []
            else: theta_in_topside2 = theta_topside2[0]

            T_topside1 = self.T_top(inputTension,theta_in_topside1,theta_topside1)

            if len(T_topside1) == 0: restingInput = inputTension
            else: restingInput = T_topside1[-1]
            T_resting = self.T_bottom(restingInput,theta_in_resting,theta_resting)
            
            if len(T_resting) == 0: topside2Input = inputTension
            else: topside2Input = T_resting[-1]
            T_topside2 = self.T_top(topside2Input,theta_in_topside2,theta_topside2)

            return np.hstack(( T_topside1, T_resting, T_topside2 )).ravel() #Combining vectors into one
        else:
            raise NameError("An unexpected number of roots found for hang-off value")

    def plotSideGeometry(self,p0,col): #Overwrite method in Section class
        super().plotSideGeometry(p0,col)
        valid_theta_crits = self.theta_crits[ (self.theta_crits > self.theta_in) & (self.theta_crits < self.theta_out) ]#Check if theta_crits are in range
        zeroVec = np.zeros(3)
        hangoffPoints = [zeroVec for i in range(len(valid_theta_crits))]
        globalCenter = rotate2global(np.array([0,0,self.R]),self.csYaw,self.csPitch)

        for i,theta_c in enumerate(valid_theta_crits):
            hangoffPoints[i] = p0 + globalCenter + rotate2global(np.array([0,0,-self.R]),self.csYaw,-theta_c)
        hangoffPoints_x = [hp[0] for hp in hangoffPoints]
        hangoffPoints_z = [hp[2] for hp in hangoffPoints]
        plt.plot(hangoffPoints_x ,hangoffPoints_z,"kx") #Hangoff points

    def plotTopGeometry(self,p0,col): #Overwrite method in Section class
        super().plotTopGeometry(p0,col)
        valid_theta_crits = self.theta_crits[ (self.theta_crits > self.theta_in) & (self.theta_crits < self.theta_out) ]#Check if theta_crits are in range
        zeroVec = np.zeros(3)
        hangoffPoints = [zeroVec for i in range(len(valid_theta_crits))]
        globalCenter = rotate2global(np.array([0,0,self.R]),self.csYaw,self.csPitch)

        for i,theta_c in enumerate(valid_theta_crits):
            hangoffPoints[i] = p0 + globalCenter + rotate2global(np.array([0,0,-self.R]),self.csYaw,-theta_c)
        hangoffPoints_x = [hp[0] for hp in hangoffPoints]
        hangoffPoints_y = [hp[1] for hp in hangoffPoints]
        plt.plot(hangoffPoints_x ,hangoffPoints_y,"kx") #Hangoff points

    def getRotatedEndPoint(self):
        return super().getRotatedEndPoint()

class TopBend(VerticalBend):
    def __init__(self,isWet,type,R,theta_in_deg,theta_out_deg,psi_in_deg):
        self.csPitch = np.deg2rad(-theta_in_deg) #Pitch of local coordinate system¨
        super().__init__(isWet,type,R,theta_in_deg,theta_out_deg,psi_in_deg)
    
    def getUnrotatedGeometry(self):
        deltaThetaVec = (self.thetaVec - self.theta_in)
        zeroVec = np.zeros(3)
        unrotPoints = [zeroVec for i in range(len(deltaThetaVec))]
        for i,dTheta in enumerate(deltaThetaVec):
            unrotPoints[i] = np.array([0,0,-self.R]) + rotate2global(np.array([0,0,self.R]),0,-dTheta) #Pitch = -theta
        return unrotPoints
    
    def getRotatedGeometry(self):
        zeroVec = np.zeros(3)
        rotPoints = [zeroVec for i in range(len(self.unrotatedPoints))]
        for i,p in enumerate(self.unrotatedPoints):
            rotPoint = rotate2global(p,self.csYaw,self.csPitch)
            #rotPoints[i] = np.array([ - rotPoint[0], rotPoint[1], rotPoint[2]]) #Mirroring about y axis
            rotPoints[i] = rotate2global(rotPoint,np.pi,0) #Mirror by half turn around z-axis
        return rotPoints
    
    def T_vec(self,inputTension):
        A = self.rho*self.R*g/(self.mu**2+1)
        B = self.mu**2 - 1
        C = self.mu*2
        exp_term = np.exp(self.mu*(self.thetaVec - self.theta_in))

        T = inputTension*(exp_term) + \
                 + A*( B*( -np.cos(self.thetaVec) + np.cos(self.theta_in)*exp_term) + \
                       C*(np.sin(self.thetaVec) - np.sin(self.theta_in)*exp_term) )
        return T

class HorizontalBend(Section):
    def __init__(self,isWet,type,R,yaw_in_deg,yaw_out_deg):
        self.csPitch = 0
        self.csYaw = np.deg2rad(yaw_in_deg)
        self.psi_in = np.deg2rad(yaw_in_deg)
        self.psi_out = np.deg2rad(yaw_out_deg)
        self.psiVec = np.linspace(self.psi_in,self.psi_out,GP_BEND)
        self.R = R
        self.theta_in = 0
        self.theta_out = 0
        super().__init__(isWet,type)
    
    def getUnrotatedGeometry(self):
        deltaPsiVec = (self.psiVec - self.psi_in)
        zeroVec = np.zeros(3)
        unrotPoints = [zeroVec for i in range(len(deltaPsiVec))]
        if (self.psi_out - self.psi_in > 0): #CounterClockwise turn
            unrotatedCenterPoint = np.array([0,self.R,0]) #In local x'-y' coordinate system
            unrotatedPoint = np.array([0,-self.R,0]) #In local x~ - y~ coordinate system
        else: 
            unrotatedCenterPoint = np.array([0,-self.R,0])
            unrotatedPoint = np.array([0,self.R,0])

        for i,dPsi in enumerate(deltaPsiVec):
            unrotPoints[i] = unrotatedCenterPoint + rotate2global(unrotatedPoint,dPsi,self.csPitch) #Pitch = -theta
        return unrotPoints

    def T_vec(self, inputTension):
        return inputTension*np.exp(self.mu*np.abs(self.psiVec - self.psi_in))

    def getLvec(self): #Length along arc as a function of thetaVec
        return np.abs(((self.psiVec-self.psi_in)*self.R))
    
    def length(self):
        return (self.getLvec()[-1])

class Pull_in:
    def __init__(self, inputTension, sections = []):
        self.inputTension = inputTension
        self.sections = sections
        #self.x0s, self.y0s, self.z0s = self.globalCoordinates()
        self.p0s = self.globalCoordinates()
        self.L,self.T = self.T_L()
        self.endYaw = self.getEndYaw()

    def addSection(self, s):
        self.sections = np.append(self.sections,s)
        self.p0s = self.globalCoordinates()
        self.L,self.T = self.T_L()
        self.endYaw = self.getEndYaw()

    def updateGlobalVariables(self):
        for sec in self.sections:
            sec.rho = determineWeight(sec.isWet)
            sec.mu = determineFrictionFactor(sec.isWet,sec.type)
        self.__init__(T_INPUT,self.sections)
    
    def updateSection(self,sec,index):
        self.sections[index] = sec
        self.__init__(T_INPUT,self.sections)

    def deleteSection(self,index):
        newSecs = np.delete(self.sections,index)
        self.__init__(T_INPUT,newSecs)
    
    def getEndYaw(self):
        if len(self.sections) > 0:
            dBetaVec = np.zeros(len(self.sections))
            isHorBend = [(type(self.sections[i]) == HorizontalBend) for i in range(len(self.sections))]
            for i,s in enumerate(self.sections): #Calculating out-in angle in horizontal direction
                if isHorBend[i]: dBetaVec[i] = s.psi_out - s.psi_in
                else: dBetaVec[i] = 0
            return np.sum(dBetaVec)
        else:
            return 0

    def sectionSeparatorLengths(self):
        individualLenghts = np.array([s.length() for s in self.sections]) #use length() function for every element in sections
        individualLenghts = np.concatenate([[0],individualLenghts])
        return np.cumsum(individualLenghts)
    
    def calculateMinWinchTension(self):
        T_cum = self.inputTension #Cumulative tension required
        for s in self.sections:
            T_cum = s.getMinOutputTension(T_cum)
        return T_cum

    def T_L(self): #Caclulate minimum tension along cable in sequence
        L_vec_global = np.array([0])
        T_vec_global = np.array([self.inputTension]) #Cumulative tension required
        self.T_inputs = np.array(np.zeros(len(self.sections))) #Tension at section inputs
        self.T_outputs = np.array(np.zeros(len(self.sections))) #Tension at section outputs
        for i,s in enumerate(self.sections):
            L_vec_global = np.concatenate([L_vec_global,L_vec_global[-1] + s.getLvec()])
            T_input = T_vec_global[-1]
            self.T_inputs[i] = T_input
            T_sec = s.T_vec(T_input)
            self.T_outputs[i] = T_sec[-1]
            T_vec_global = np.concatenate([T_vec_global,T_sec])
        
        return L_vec_global,T_vec_global

    def globalCoordinates(self): #Compute startpoint of all sections
        zeroVec = np.zeros(3)
        p0s = [zeroVec for i in range(len(self.sections))]
        for i in range(1,len(self.sections)):
            p0s[i] = p0s[i-1] + self.sections[i-1].getRotatedEndPoint()
        return p0s

    def plotSideGeometry(self): #From side
        index = 0
        for s in self.sections:
            if (index % 2) == 0: isOdd = False
            else: isOdd = True #Odd number section
            s.plotSideGeometry(self.p0s[index],isOdd) #Call local plotting functon with global coordinates
            index = index + 1
        self.sections[0].plotInputTensionArrowSide(self.p0s[0]) #Input tension vector
        self.sections[-1].plotOutputTensionArrowSide(self.p0s[-1]) #Output tension vector. Inputtension for scale
        plt.xlabel("x [m]")
        plt.ylabel("z [m]")

    def plotTopGeometry(self): #From side
        #plt.figure(3)
        index = 0
        for s in self.sections:
            if (index % 2) == 0: isOdd = False
            else: isOdd = True #Odd number section
            s.plotTopGeometry(self.p0s[index],isOdd) #Call local plotting functon with global coordinates
            index = index + 1
        self.sections[0].plotInputTensionArrowTop(self.p0s[0]) #Input tension vector
        self.sections[-1].plotOutputTensionArrowTop(self.p0s[-1]) #Output tension vector. Inputtension for scale
        plt.xlabel("x [m]")
        plt.ylabel("y [m]")
    
    def plot3dGeometry(self):

        plt.figure(5)
        ax = plt.axes(projection='3d')
        index = 0
        for s in self.sections:
            if (index % 2) == 0: col = "c" #Even number section
            else: col = "b" #Odd number section
            s.plot3dGeometry(self.p0s[index],ax,col) #Call local plotting functon with global coordinates
            index = index + 1
        #self.sections[0].plotInputTensionArrowTop(self.p0s[0]) #Input tension vector
        #self.sections[-1].plotOutputTensionArrowTop(self.p0s[-1]) #Output tension vector. Inputtension for scale
        plt.xlabel("x [m]")
        plt.ylabel("y [m]")

    def tensionHangofPoints(self):
        L_points = np.array([])
        sepLengths = self.sectionSeparatorLengths()
        for i in range(len(self.sections)):
            s = self.sections[i]
            if isinstance(s,BottomBend):
                L_points = np.concatenate([L_points,sepLengths[i] + s.critLenghts()])
        return L_points
    
    def plotMinTensionAlongCable(self):
        #print(len(self.L),len(self.T))
        plt.figure(3)
        plt.plot(self.L,self.T,"k")
        plt.xlabel("L [m]")
        plt.ylabel("T(L) [N]")
        [plt.axvline(L_ho, color='g',linestyle="-.") for L_ho in self.tensionHangofPoints()] #Plot vertical lines for hangoffs
        [plt.axvline(sepLen, color='k',linestyle="--") for sepLen in self.sectionSeparatorLengths()] #Plot vertical sep lines
        for L_ho in self.tensionHangofPoints():
            text(L_ho, max(self.T)*0.98, r'$L_{ho}$ = %.2f' % L_ho, rotation=90, va='top', ha='right',color="g") #Text for hangoff point vlines
        plt.title("Tension along cable")

#endregion
pi = Pull_in(T_INPUT,np.array([]))
#GUI
#region

# Output Widgets
#region
outText = widgets.Output(layout = Layout(width = '90%',border = '1px solid grey'))
outGeoSide = widgets.Output(layout = Layout(width = '50%',border = '1px solid grey'))
outGeoTop = widgets.Output(layout = Layout(width = '50%',border = '1px solid grey'))
outTension = widgets.Output(layout = Layout(width = '50%',border = '1px solid grey'))
#endregion

# Global parameter widgets
#region
wStyle = {'description_width': 'initial'}

rho_w_BFT = widgets.BoundedFloatText(value=RHO_WET, min=0, max=200, description = "Wet weight [kg/m]", style = wStyle)
rho_d_BFT = widgets.BoundedFloatText(value=RHO_DRY, min=0, max=200, description = "Dry weight [kg/m]", style = wStyle)
mu_wp_BFT = widgets.BoundedFloatText(value=MU_WET_PIPE, min=0, max=1, step=0.01, description = "Wet pipe friction factor [-]", style = wStyle)
mu_dp_BFT = widgets.BoundedFloatText(value=MU_DRY_PIPE, min=0, max=1, step=0.01, description = "Dry pipe friction factor [-]", style = wStyle)
mu_wr_BFT = widgets.BoundedFloatText(value=MU_WET_ROLLER, min=0, max=1, step=0.01, description = "Wet roller friction factor [-]", style = wStyle)
mu_dr_BFT = widgets.BoundedFloatText(value=MU_DRY_ROLLER, min=0, max=1, step=0.01, description = "Dry roller friction factor [-]", style = wStyle)
T_in_BIT = widgets.BoundedIntText(value=T_INPUT, min=0, max=1e6, step=100, description = "Input tension [N]", style = wStyle,layout = Layout(width = '20%'))
saveBtn = widgets.Button(description='Save parameters', disabled=True, icon='save',button_style='primary')
editGlbBtn = widgets.Button(description='Edit parameters', disabled=False, icon='edit',button_style='primary')

weightBox = widgets.VBox(children= [rho_w_BFT,rho_d_BFT],layout = Layout(width = '30%'))
frictionBox = widgets.VBox(children = [mu_wp_BFT, mu_dp_BFT, mu_wr_BFT, mu_dr_BFT],layout = Layout(width = '30%'))
buttonGlbBox = widgets.VBox(children = [editGlbBtn,saveBtn])
globalParamBox = widgets.HBox(children = [frictionBox,weightBox,T_in_BIT,buttonGlbBox])
globalParamList = [mu_wp_BFT, mu_dp_BFT, mu_wr_BFT, mu_dr_BFT, rho_w_BFT, rho_d_BFT, T_in_BIT] #For easy enabling/disabling
for w in globalParamList: w.disabled = True #Start with all parameters locked into default
#endregion

# Geometry widgets
#region
isWet_RB = widgets.RadioButtons(options = [("Wet",True),("Dry",False)],layout=Layout(width='5%'))
Material_RB = widgets.RadioButtons(options = [("Pipe",SectionType.PIPE),("Roller",SectionType.ROLLER)],style = wStyle,layout=Layout(width='7.5%'))
geometry_RB = widgets.RadioButtons(options = [("Straight",Straight),("Bottom bend",BottomBend),("Top bend",TopBend),("Horizontal bend",HorizontalBend)],layout=Layout(width='10%'))
horDisp_BFT = widgets.BoundedFloatText(value=50, min=0, max=2000, step = 1, description = "Delta x' [m]",layout=Layout(max_width='50%'))
vertDisp_BFT = widgets.BoundedFloatText(value=5, min=-100 , max=100, description = "Delta z' [m]",layout=Layout(max_width='50%'))
rad_BFT = widgets.BoundedFloatText(value=5, min=0 ,max=2000, description = "Bend radius [m]", style = wStyle)
thetaRange_IRS= widgets.IntRangeSlider(value=[-45, 45], min=-90, max=90, step=1, description = "Pitch range",style = wStyle)
psiRange_IS = widgets.IntSlider(value = 0, min=-90, max=90, step=5, description = "Yaw range",style = wStyle)
addSecBtn = widgets.Button(description='Add section', disabled=False,icon='check',button_style='success')
editSecBtn = widgets.Button(description='Edit section', disabled=True, icon='edit',button_style='primary')
deleteScnBtn = widgets.Button(description='Delete', disabled= True, icon='close',button_style='danger')
updateScnBtn = widgets.Button(description='Update', disabled= True, icon='refresh',button_style='primary')
editSecNum_BIT = widgets.BoundedIntText(description= "Section #", disabled = True, value=1, min=1 , max = 1,layout=Layout(max_width='75%'),style = wStyle)
exportBtn = widgets.Button(description='Export', disabled=False, icon='save', button_style='success')
axisEqCB = widgets.Checkbox(description='Axes equal', disabled=False, value=True, indent=False)

editButtonBox = widgets.VBox(children = [editSecNum_BIT, widgets.HBox(children = [updateScnBtn, deleteScnBtn])])
buttonSecBox = widgets.HBox(children = [addSecBtn,editSecBtn,editButtonBox],layout=Layout(width="45%"))

editList = [editSecNum_BIT,deleteScnBtn,updateScnBtn]

secVarOptions = widgets.VBox(children= [horDisp_BFT, vertDisp_BFT],layout=Layout(width='30%')) #Options for adding Straight section
sectionBox = widgets.HBox(children = [isWet_RB,Material_RB, geometry_RB, secVarOptions, buttonSecBox],style = wStyle)
sectionBox.layout.border = '1px solid grey'

sectionHBox = widgets.HBox(children = [isWet_RB, Material_RB, geometry_RB, horDisp_BFT, vertDisp_BFT]) #Only for iterative purposes
#endregion

# Collect widgets
#region
tabTitles = ["Geometry","Global parameters"]
tab = widgets.Tab(children = [sectionBox,globalParamBox])
tab.layout.width = '100%'
[tab.set_title(i, title) for i, title in enumerate(tabTitles)]

topBox = widgets.HBox(children = [tab])
middleBox = widgets.HBox(children = [outText,widgets.VBox(children = [exportBtn,axisEqCB],layout=Layout(width='10%'))])
bottomBox = widgets.HBox(children = [outGeoSide,outGeoTop,outTension])
ui = widgets.VBox(children = [topBox, middleBox, bottomBox ])
#display(outDebug)
display(ui)
#endregion

#Callback functions
#region
def changeGeometrySpecificationUI(obj):
    if geometry_RB.value == Straight:
        secVarOptions.children = [horDisp_BFT, vertDisp_BFT]
        sectionHBox.children = [ isWet_RB, Material_RB, geometry_RB, horDisp_BFT, vertDisp_BFT] 
    elif geometry_RB.value == HorizontalBend:
        secVarOptions.children = [rad_BFT, psiRange_IS]
        sectionHBox.children = [ isWet_RB, Material_RB, geometry_RB, rad_BFT, psiRange_IS] 
    else:
        secVarOptions.children = [rad_BFT, thetaRange_IRS]
        sectionHBox.children = [isWet_RB, Material_RB, geometry_RB, rad_BFT, thetaRange_IRS]

def compileSectionFromGUI(): #Compiles a Section object with widget values
    yaw = np.rad2deg(pi.endYaw) #So that input is degrees
    if geometry_RB.value == Straight: 
        sec = Straight(isWet_RB.value,Material_RB.value,horDisp_BFT.value,vertDisp_BFT.value,yaw)
    elif geometry_RB.value == BottomBend: 
        sec = BottomBend(isWet_RB.value,Material_RB.value,rad_BFT.value,thetaRange_IRS.value[0],thetaRange_IRS.value[1],yaw)
    elif geometry_RB.value == TopBend: 
        sec = TopBend(isWet_RB.value,Material_RB.value,rad_BFT.value,thetaRange_IRS.value[0],thetaRange_IRS.value[1],yaw)
    elif geometry_RB.value == HorizontalBend: 
        sec = HorizontalBend(isWet_RB.value,Material_RB.value,rad_BFT.value,yaw,psiRange_IS.value)
    return sec

def addSection_callback(obj):
    s = compileSectionFromGUI()
    pi.addSection(s)
    editSecBtn.disabled = False
    updateSectionOutput()
    updateGUIplots()

def editSection(obj):
    editSecBtn.disabled = True
    addSecBtn.disabled = True
    deleteScnBtn.disabled = False
    updateScnBtn.disabled = False
    editSecNum_BIT.unobserve(adjustWidgetsWithSectionSpecifyer)
    editSecNum_BIT.max = len(pi.sections)
    editSecNum_BIT.disabled = False
    editSecNum_BIT.observe(adjustWidgetsWithSectionSpecifyer)

def updateSection_callback(obj):
    editSecNum_BIT.unobserve(adjustWidgetsWithSectionSpecifyer) #To avoid observe callback function to trigger by disabling editSecNum_BIT
    for w in editList: w.disabled = True
    editSecNum_BIT.observe(adjustWidgetsWithSectionSpecifyer)    
    addSecBtn.disabled = False
    editSecBtn.disabled = False
    secIndex = editSecNum_BIT.value - 1
    replacementSec = compileSectionFromGUI()
    pi.updateSection(replacementSec,secIndex)
    updateSectionOutput()
    updateGUIplots()

def deleteSection_callback(obj):
    #editSecNum_BIT.unobserve(adjustWidgetsWithSectionSpecifyer)
    for w in editList: w.disabled = True
    addSecBtn.disabled = False
    editSecBtn.disabled = False
    secIndex = editSecNum_BIT.value - 1
    pi.deleteSection(secIndex)
    updateSectionOutput()
    updateGUIplots()

def adjustWidgetsWithSectionSpecifyer(obj):
    secIndex = editSecNum_BIT.value - 1
    sec = pi.sections[secIndex]
    isWet_RB.value = sec.isWet
    Material_RB.value = sec.type
    geometry_RB.value = type(sec)
    if type(sec) == Straight:
        horDisp_BFT.value = sec.horDisp
        vertDisp_BFT.value = sec.vertDisp
    else:
        rad_BFT.value = sec.R
        thetaRange_IRS.value = [np.rad2deg(sec.theta_in),np.rad2deg(sec.theta_out)]

def saveGlobalParameters(obj):
    global RHO_WET, RHO_DRY, MU_WET_PIPE,MU_DRY_PIPE, MU_WET_ROLLER, MU_DRY_ROLLER, T_INPUT
    RHO_WET = rho_w_BFT.value
    RHO_DRY = rho_d_BFT.value
    MU_WET_PIPE = mu_wp_BFT.value
    MU_DRY_PIPE = mu_dp_BFT.value
    MU_WET_ROLLER = mu_wr_BFT.value
    MU_DRY_ROLLER = mu_dr_BFT.value
    T_INPUT = T_in_BIT.value
    saveBtn.disabled = True
    editGlbBtn.disabled = False
    for w in globalParamList: w.disabled = True
    pi.updateGlobalVariables() #Re-initializes itself
    updateGUIplots()
    updateSectionOutput()
    
def editGlobalParamaters(obj):
    saveBtn.disabled = False
    editGlbBtn.disabled = True
    for w in globalParamList: w.disabled = False
            
def updateSectionOutput():
    #template = "{0:2} | {1:14} | {2:3} | {3:6} | {4:6} | {5:10} | {6:10} | {7:17} | {8:17}" #{Num,coloumnWidth}
    str = generateTextOutput()
    with outText: clear_output()
    #with outText: print(template.format("#", "TYPE", "W/D", "P/R", "MU [-]", "RHO [kg/m]","LENGTH [m]", "Input tension [N]", "Output tension [N]"))
    with outText: print(str)

def generateTextOutput():
    template = "{0:2} | {1:17} | {2:3} | {3:6} | {4:6} | {5:10} | {6:10} | {7:17} | {8:17}" #{Num,coloumnWidth}
    str = template.format("#", "TYPE", "W/D", "P/R", "MU [-]", "RHO [kg/m]","LENGTH [m]", "Input tension [N]", "Output tension [N]") + "\n"
    for i,sec in enumerate(pi.sections):
        T_in = int(pi.T_inputs[i])
        T_out = int(pi.T_outputs[i])
        length = round(sec.length(),1)

        if isinstance(sec,Straight): secTypeStr = "Straight"
        elif isinstance(sec,BottomBend): secTypeStr = "BottomBend"
        elif isinstance(sec,TopBend): secTypeStr = "TopBend"
        elif isinstance(sec,HorizontalBend): secTypeStr = "HorizontalBend"

        if sec.isWet: isWetStr = "Wet"
        else: isWetStr = "Dry"

        if sec.type==SectionType.PIPE: typeStr = "Pipe"
        elif sec.type==SectionType.ROLLER: typeStr = "Roller"

        str = str + template.format(i+1,secTypeStr, isWetStr, typeStr, sec.mu, sec.rho, length, T_in, T_out) + "\n"
    return str

def updateGUIplots():
    if len(pi.sections) > 0:
        with outGeoSide: clear_output()
        with outGeoTop: clear_output()
        with outTension: clear_output()
        with outGeoSide:
            pi.plotSideGeometry()
            if axisEqCB.value: plt.gca().axis("equal")
            else: plt.gca().axis("auto")
            plt.gcf().set_dpi(300)
            plt.show()
        with outGeoTop:
            pi.plotTopGeometry()
            #pi.plot3dGeometry()
            if axisEqCB.value: plt.gca().axis("equal")
            else: plt.gca().axis("auto")
            plt.gcf().set_dpi(300)
            plt.show()
        with outTension: 
            pi.plotMinTensionAlongCable()
            plt.gcf().set_dpi(300)
            plt.show()

def textOutput2figure():
    str = generateTextOutput()
    plt.figure(5)
    plt.figtext(0.05, 0.95 , str , va='top', bbox = dict(boxstyle='round', facecolor='cyan', alpha=0.3))
    plt.axis('off')

def sectionTableOutput2figure(): #Generate table with section info
    column_labels = ["#", "TYPE", "W/D", "P/R","R",
                    r"$\theta_{in}$"+"\n[deg]",r"$\theta_{out}$"+"\n[deg]",r"$\psi_{in}$"+"\n[deg]",r"$\psi_{out}$"+"\n[deg]"]
    numVec = [i for i in range(1,len(pi.sections)+1)]
    typeVec = ["Straight"*(type(s)==Straight) + "Bottom\nBend"*(type(s)==BottomBend) + "Top\nBend"*(type(s)==TopBend) + 
                "Horizontal\nBend"*(type(s)==HorizontalBend) for s in pi.sections]
    isWetVec = [(s.isWet==True)*"Wet" + (s.isWet==False)*"Dry"  for s in pi.sections]
    sectionTypeStringVec = ["Pipe"*(s.type==SectionType.PIPE) + "Roller"*(s.type==SectionType.ROLLER) for s in pi.sections]
    radVec = [s.R for s in pi.sections]
    inputThetaVec = [round(np.rad2deg(s.theta_in)) for s in pi.sections]
    outputThetaVec = [round(np.rad2deg(s.theta_out)) for s in pi.sections]
    inputPsiVec = [round(np.rad2deg(s.psi_in)) for s in pi.sections]
    outputPsiVec = [round(np.rad2deg(s.psi_out)) for s in pi.sections]
    notTransposedList = [numVec,typeVec,isWetVec,sectionTypeStringVec, radVec, inputThetaVec, outputThetaVec,inputPsiVec, outputPsiVec]
    cell_text = [list(i) for i in zip(*notTransposedList)] #Transposing list by making lists out of every i'th unziped item in individual lists. Use instead of np.transpose() to keep significant digits
    #cell_text = np.transpose([numVec,typeVec,isWetVec,sectionTypeStringVec, radVec, inputThetaVec, outputThetaVec,inputPsiVec, outputPsiVec]) #Arrange vector data to cell text matrix
    plt.figure(5)
    plt.table(cellText=cell_text, colLabels=column_labels, bbox = [0.0, 0.0, 1.0, 1.0], cellLoc='center')
    plt.title("Input")
    plt.axis('off')

def calcTableOutput2figure(): #Generate table with calculations
    column_labels = ["#", r"$\mu$"+"\n[-]", r"$\rho$"+"\n[kg/m]", "L\n[m]",r"$m$"+"\n[kg]","Input\ntension\n[N]", "Section\ncontribution\n[N]", "Output\ntension\n[N]"]
    numVec = [i for i in range(1,len(pi.sections)+1)]
    muVec = [round(s.mu,2) for s in pi.sections]
    rhoVec = [round(s.rho,1) for s in pi.sections]
    lengthVec = [round(s.length(),1) for s in pi.sections]
    massVec = [int(s.rho*s.length()) for s in pi.sections]
    inputTensionVec = [int(pi.T_inputs[i])  for i,sec in enumerate(pi.sections)]
    outputTensionVec = [int(pi.T_outputs[i]) for i,sec in enumerate(pi.sections)]
    sectionTensionVec = np.array(outputTensionVec) - np.array(inputTensionVec)
    notTransposedList = [numVec, muVec, rhoVec, lengthVec, massVec, inputTensionVec, sectionTensionVec, outputTensionVec]
    cell_text = [list(i) for i in zip(*notTransposedList)] #Transposing list by making lists out of every i'th unziped item in individual lists. Use instead of np.transpose() to keep significant digits

    plt.figure(6)
    plt.table(cellText=cell_text, colLabels=column_labels, bbox = [0.0, 0.0, 1.0, 1.0], cellLoc='center')
    plt.title("Output")
    plt.axis('off')

def export2pdf(obj):
    filename = 'pull_in_calc.pdf'
    pp = PdfPages(filename)

    #textOutput2figure()
    sectionTableOutput2figure()
    pp.savefig()

    calcTableOutput2figure()
    pp.savefig()

    pi.plotMinTensionAlongCable()
    plt.gcf().set_dpi(500)
    pp.savefig()

    pi.plotSideGeometry()
    if axisEqCB.value: plt.gca().axis("equal")
    else: plt.gca().axis("auto")
    plt.gcf().set_dpi(500)
    pp.savefig()

    pi.plotTopGeometry()
    if axisEqCB.value: plt.gca().axis("equal")
    else: plt.gca().axis("auto")
    plt.gcf().set_dpi(500)
    pp.savefig()

    pp.close()

    with outText: display(FileLink('Pull_in_calc.pdf'))

def axisEqualCB_callback(obj):
    updateGUIplots()

addSecBtn.on_click(addSection_callback)
saveBtn.on_click(saveGlobalParameters)
editGlbBtn.on_click(editGlobalParamaters)
editSecBtn.on_click(editSection)
updateScnBtn.on_click(updateSection_callback)
deleteScnBtn.on_click(deleteSection_callback)
editSecNum_BIT.observe(adjustWidgetsWithSectionSpecifyer)
geometry_RB.observe(changeGeometrySpecificationUI)
exportBtn.on_click(export2pdf)
axisEqCB.observe(axisEqualCB_callback,names='value') #Only call function when value changes. If names='value' not specified, function will be called several times
#endregion

#endregion


In [27]:
#!pipreqs --force
#!pip3 install nbconvert
#!jupyter nbconvert --to script Pull_in_calculations.ipynb
#!cd reqs
#!pipreqs --force
#!pipreqs \Users\thaddal\OneDrive - Nexans\Documents\Python Scripts\Pull_in_calculations\

#FileLinks('.')

INFO: Successfully saved requirements file in c:\Users\thaddal\OneDrive - Nexans\Documents\Python Scripts\Pull_in_calculations\requirements.txt
